In [14]:
import tensorflow as tf
import numpy as np

# The computatioonal Graph

In [15]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also ft.float32 implicitly
print(node1, node2)

(<tf.Tensor 'Const_2:0' shape=() dtype=float32>, <tf.Tensor 'Const_3:0' shape=() dtype=float32>)


In [16]:
sess = tf.Session()
node3 = tf.add(node1, node2)
print('note3:', node3)
print('sess.run(node3):', sess.run(node3))

('note3:', <tf.Tensor 'Add_4:0' shape=() dtype=float32>)
('sess.run(node3):', 7.0)


In [17]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a / b

print(sess.run(adder_node, {b: 2, a:1}))

0.5


In [28]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [29]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
x_data = np.arange(4)
print(sess.run(loss, {x: x_data, y: [0, -1, -2, -3]}))

19.34


In [30]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


# tf.train API

In [31]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
sess.run(init) # reset values to incorrect defaults
for i in range(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


# Complete program

In [37]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3])
b = tf.Variable([-.3])
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y))
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# train data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

# training leep
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})

curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print('W: %s b: %s loss: %s' %(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


# tf.estimater

In [45]:
import numpy as np
import tensorflow as tf

# Declare list of features
feature_columns = [tf.feature_column.numeric_column('x', shape=[1])]
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

input_fn = tf.estimator.inputs.numpy_input_fn(
    {'x': x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'x': x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'x': x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

estimator.train(input_fn=input_fn, steps=1000)

train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print('train metrics: %r' %train_metrics)
print('eval metrics: %r' %eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tmpA7XrSX', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpA7XrSX/model.ckpt.
INFO:tensorflow:loss = 9.0, step = 1
INFO:tensorflow:global_step/sec: 1052.93
INFO:tensorflow:loss = 0.118896, step = 101 (0.096 sec)
INFO:tensorflow:global_step/sec: 1152.45
INFO:tensorflow:loss = 0.0164067, step = 201 (0.087 sec)
INFO:tensorflow:global_step/sec: 1161.2
INFO:tensorflow:loss = 0.00139166, step = 301 (0.086 sec)
INFO:tensorflow:global_step/sec: 1141.51
INFO:tensorflow:loss = 7.6898e-05, step = 401 (0.088 sec)
INFO:tensorflow:global_step/sec: 1138.95
INFO:tensorflow:loss = 5.2261e-05, step = 501 (0.088 sec)
INFO:tensorflow:g